The goal of this file is to load all flight, fuel, airport, and trajectory data, engineer a feature set by aggregating time-series trajectory statistics (like mean altitude and speed) for each individual fuel-burn interval, and then use Optuna to tune and train a LightGBM regression model to predict the fuel consumed in each interval.

This script provides a complete baseline, but the feature engineering method used is **extremely** inefficient.

Mix ai generated gemini 2.5 pro and thibault lootvoet + copilot

LGBM is mega slow in wsl

In [1]:
import datetime
import glob
import os
import sys

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
def load_data():
    """Loads all parquet files and performs initial type conversion."""
    print("Loading data...")
    base_dir = "../.."
    try:
        # Load the single-file datasets
        flights_df = pd.read_parquet(f"{base_dir}/data/flightlist_train.parquet")
        fuel_df = pd.read_parquet(f"{base_dir}/data/fuel_train.parquet")
        airports_df = pd.read_parquet(f"{base_dir}/data/apt.parquet")

        print("Loaded flight list, fuel, and airport data.")

        # --- Load all trajectory files from the 'flights_train' directory ---

        traj_path = f"{base_dir}/data/flights_train"

        # Check if the directory exists
        if not os.path.isdir(traj_path):
            raise FileNotFoundError(
                f"Trajectory directory not found: '{traj_path}'. "
                "Please ensure it's in the same location as the script."
            )

        # Use glob to find all .parquet files in the directory
        all_traj_files = glob.glob(os.path.join(traj_path, "*.parquet"))
        # all_traj_files = all_traj_files[0:200] # for faster testing, to be removed later

        if not all_traj_files:
            raise FileNotFoundError(
                f"No .parquet files found in '{traj_path}' directory."
            )

        print(
            f"Found {len(all_traj_files)} trajectory files. Loading and concatenating..."
        )

        # Create a list of dataframes, one for each file
        traj_dfs = [pd.read_parquet(f) for f in all_traj_files]

        # Concatenate all of them into a single dataframe
        traj_df = pd.concat(traj_dfs, ignore_index=True)

        print("Trajectory data concatenated successfully.")

        # --- Data Preprocessing and Cleaning ---
        print("Preprocessing data...")

        # Convert all relevant timestamps to datetime objects
        flights_df["takeoff"] = pd.to_datetime(flights_df["takeoff"])
        flights_df["landed"] = pd.to_datetime(flights_df["landed"])

        traj_df["timestamp"] = pd.to_datetime(traj_df["timestamp"])

        fuel_df["start"] = pd.to_datetime(fuel_df["start"])
        fuel_df["end"] = pd.to_datetime(fuel_df["end"])

        # "Quick and Dirty" cleaning: Drop flights with missing aircraft_type
        flights_df = flights_df.dropna(subset=["aircraft_type"])

        return flights_df, traj_df, fuel_df, airports_df

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print(
            "Please make sure all files (apt.parquet, flightlist_train.parquet, fuel_train.parquet) "
            "and the directory (flights_train/) are in the same location."
        )
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred during data loading: {e}")
        print("You may need to install a parquet engine, e.g.: pip install pyarrow")
        sys.exit(1)

In [15]:
def create_features(
    flights_df: pd.DataFrame,
    traj_df: pd.DataFrame,
    fuel_df: pd.DataFrame,
    airports_df: pd.DataFrame,
    to_display: bool = False,
) -> tuple[pd.DataFrame, pd.Series, list]:
    """
    Engineers a model-ready feature set (X) and target (y) from the raw data.

    The core logic is built around the `fuel_df`, where each row represents
    a single fuel-burn interval and becomes one sample for the model.

    This function performs three main steps:
    1.  Merges Static Data: Enriches each fuel interval with static
        data from `flights_df` (e.g., aircraft_type) and `airports_df`
        (e.g., origin/destination elevation).
    2.  Filters & Aggregates Time-Series: For each interval, it finds all
        trajectory points from `traj_df` that fall *within* its [start, end]
        timestamp. It then aggregates these points to create summary
        statistics (e.g., mean_altitude, max_TAS, n_points).
    3.  Cleans & Finalizes: Calculates delta features (e.g., altitude_change),
        handles missing values, and formats the final feature matrix `X`
        and target vector `y`.

    Returns:
        X (pd.DataFrame): DataFrame of features for the model.
        y (pd.Series): Series of the target variable (fuel_kg).
        categorical_features (list): List of column names to be
                                     treated as categorical.
    """
    print("Engineering features...")

    # 3.1: Create base table from fuel data
    # Each row in fuel_df is our target sample
    data = fuel_df.copy()

    # 3.2: Add flight-level features
    data = pd.merge(
        data,
        flights_df[["flight_id", "aircraft_type", "origin_icao", "destination_icao"]],
        on="flight_id",
        how="left",
    )

    # 3.3: Add airport features (elevation)
    airports_df_simple = airports_df[["icao", "elevation"]].rename(
        columns={"elevation": "origin_elevation"}
    )
    data = pd.merge(
        data, airports_df_simple, left_on="origin_icao", right_on="icao", how="left"
    )
    airports_df_simple = airports_df_simple.rename(
        columns={"origin_elevation": "dest_elevation"}
    )
    data = pd.merge(
        data,
        airports_df_simple,
        left_on="destination_icao",
        right_on="icao",
        how="left",
    )

    # 3.4: Engineer interval-based trajectory features

    # Give each interval a unique ID to group by
    data["interval_id"] = data.index

    # Merge all trajectory points with all intervals for the same flight
    # This creates a large table, but allows vectorized filtering.
    merged_traj = pd.merge(
        traj_df, data[["flight_id", "start", "end", "interval_id"]], on="flight_id"
    )

    # Filter: Keep only trajectory points that fall *within* the [start, end] of a fuel interval
    filtered_traj = merged_traj[
        (merged_traj["timestamp"] >= merged_traj["start"])
        & (merged_traj["timestamp"] <= merged_traj["end"])
    ].copy()

    # 3.5: Aggregate trajectory features by interval
    print("Aggregating trajectory features...")

    # Sort by time to get first/last points correctly
    filtered_traj.sort_values(by=["interval_id", "timestamp"], inplace=True)

    # Simple aggregates
    agg_features = filtered_traj.groupby("interval_id").agg(
        mean_altitude=("altitude", "mean"),
        max_altitude=("altitude", "max"),
        mean_TAS=("TAS", "mean"),
        max_TAS=("TAS", "max"),
        mean_groundspeed=("groundspeed", "mean"),
        mean_vertical_rate=("vertical_rate", "mean"),
        n_points=("timestamp", "count"),
    )

    # Get features from the *first* and *last* point in the interval
    first_points = filtered_traj.groupby("interval_id").first()
    last_points = filtered_traj.groupby("interval_id").last()

    delta_features = pd.DataFrame(index=agg_features.index)
    delta_features["altitude_change"] = (
        last_points["altitude"] - first_points["altitude"]
    )
    delta_features["time_in_interval_s"] = (
        last_points["timestamp"] - first_points["timestamp"]
    ).dt.total_seconds()

    # 3.6: Merge aggregated features back into the main table
    data = data.merge(agg_features, on="interval_id", how="left")
    data = data.merge(delta_features, on="interval_id", how="left")

    # 3.7: Add simple features
    # This is the 'reported' duration, which is a key predictor
    data["interval_duration_s"] = (data["end"] - data["start"]).dt.total_seconds()

    # --- 4. Final Processing ---
    print("Finalizing dataset...")

    # Define feature set
    # We must convert aircraft_type to a category for LightGBM
    data["aircraft_type"] = data["aircraft_type"].astype("category")

    # "Quick and Dirty" cleaning:
    # Drop rows where we have no fuel target
    data = data.dropna(subset=["fuel_kg"])
    # Drop rows where interval duration is zero or negative
    data = data[data["interval_duration_s"] > 0]
    # Drop intervals that had no trajectory points (n_points is NaN after left merge)
    data = data.dropna(subset=["n_points"])

    # Fill any other remaining NaNs (e.g., missing TAS) with 0
    # A better approach would be interpolation or 0-filling only specific columns
    data["aircraft_type"] = data["aircraft_type"].astype("object")
    data["aircraft_type"] = data["aircraft_type"].fillna("UNKNOWN")
    data["aircraft_type"] = data["aircraft_type"].astype("category")

    numeric_cols = data.select_dtypes(include=np.number).columns
    data[numeric_cols] = data[numeric_cols].fillna(0)

    features = [
        "interval_duration_s",
        "aircraft_type",
        "origin_elevation",
        "dest_elevation",
        "mean_altitude",
        "max_altitude",
        "mean_TAS",
        "max_TAS",
        "mean_groundspeed",
        "mean_vertical_rate",
        "n_points",
        "altitude_change",
        "time_in_interval_s",
    ]
    categorical_features = ["aircraft_type"]
    target = "fuel_kg"

    # Filter out any rows that somehow still have NaN/Inf in features
    X = data[features]
    y = data[target]

    # Define our numeric features (which is all features except the categorical ones)
    numeric_features = [f for f in features if f not in categorical_features]

    # Apply numeric-only cleaning (replace inf, fillna)
    # ONLY to the numeric columns.
    X[numeric_features] = (
        X[numeric_features].replace([np.inf, -np.inf], np.nan).fillna(0)
    )

    print(f"Dataset finalized. Total samples: {len(X)}")
    print(f"Features: {features}")

    if to_display:
        display(X.head())
        display(y.head())
    print(f"Categorical features: {categorical_features}")

    return X, y, categorical_features

In [ ]:
def train_model(X, y, categorical_features, n_trials=50, to_load=None) -> str:
    """Sets up Optuna to find the best LightGBM hyperparameters."""

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    def objective(trial):
        """Optuna objective function to minimize."""

        ## see this for a proposition of hyperparamters https://towardsdatascience.com/gradient-boosting-to-early-stop-or-not-to-early-stop-5ea67ac09d83/
        """
        def _objective(t, dtrain, dval, early_stopping):
            params = {
                'boosting_type': t.suggest_categorical(['gbdt', 'goss']),
                'learning_rate': t.suggest_float(0.01, 0.5, log=True),
                'min_split_gain': t.suggest_float(0.00001, 2, log=True),
                'num_leaves': t.suggest_int(2, 1024, log=True),
                'max_depth': t.suggest_int(1, 15),
                'min_child_samples': t.suggest_int(2, 100, log=True),
                'bagging_freq': t.suggest_categorical([0, 1]),
                'pos_bagging_fraction': t.suggest_float(0, 1),
                'neg_bagging_fraction': t.suggest_float(0, 1),
                'reg_alpha': t.suggest_float(0.00001, 0.1, log=True),
                'reg_lambda': t.suggest_float(0.00001, 0.1, log=True),
            }
            model = lgb.train(
                **params, dtrain,
                num_boost_round=(
                    4000 if early_stopping
                    else trial.suggest_int('num_boost_rounds', 10, 4000)
                ),
                valid_sets=dval if early_stopping else None,
                callbacks=(
                    [lgb.early_stopping(stopping_rounds=100)] if early_stopping
                    else None))
        """

        # Hyperparameters to tune
        params = {
            "objective": "regression_l1",
            "metric": "rmse",
            "n_estimators": 100,  ###
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 300),
            "max_depth": trial.suggest_int("max_depth", 5, 20),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
            "verbose": -1,
            "n_jobs": -1,
            "seed": 42,
        }

        model = lgb.LGBMRegressor(**params)

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[lgb.early_stopping(100, verbose=False)],
            categorical_feature=categorical_features,
        )

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    print("\n--- Starting Optuna Hyperparameter Search ---")
    if to_load:
        study = optuna.load_study(study_name=to_load, storage="sqlite:///mydb.sqlite3")
        print(f"Loaded existing study '{to_load}' with {len(study.trials)} trials.")
    else:
        study = optuna.create_study(
            direction="minimize",
            study_name=f"LightGBM Fuel Burn Prediction {datetime.datetime.now().isoformat()}",
            storage="sqlite:///mydb.sqlite3",
            load_if_exists=True,
        )

    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print("\nOptuna search finished.")
    print(f"Best RMSE on validation set: {study.best_value:.4f}")
    print("Best hyperparameters:")
    print(study.best_params)

    # Final model evaluation
    print("\n--- Training Final Model on All Training Data ---")

    # Add n_estimators from the best trial (or use the fitted one from early stopping)
    best_params = study.best_params
    best_params["n_estimators"] = 1000  # We will use early stopping

    final_model = lgb.LGBMRegressor(
        objective="regression_l1",
        metric="rmse",
        seed=42,
        n_jobs=-1,
        verbose=-1,
        **best_params,
    )

    final_model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(100, verbose=True)],
        categorical_feature=categorical_features,
    )

    val_preds = final_model.predict(X_val)
    final_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
    print(f"\nFinal Model Validation RMSE: {final_rmse:.4f}")

    # Show feature importances
    try:
        lgb.plot_importance(final_model, max_num_features=20)
        import matplotlib.pyplot as plt

        plt.title("Feature Importances")
    except Exception as e:
        print(
            f"\nCould not plot feature importance. (Is matplotlib installed?) Error: {e}"
        )

    return study.study_name

In [ ]:
def test_model(
    X, y, categorical_features, model_to_load, storage="sqlite:///mydb.sqlite3"
):
    """Loads the best model from the Optuna study."""
    study = optuna.load_study(study_name=model_to_load, storage=storage)
    best_trial = study.best_trial
    print(f"Best trial: {best_trial.number}")
    print(f"Best value (RMSE): {best_trial.value:.4f}")
    print("Best hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")

    # Train final model with best hyperparameters
    final_model = lgb.LGBMRegressor(
        objective="regression_l1",
        metric="rmse",
        seed=42,
        n_jobs=-1,
        verbose=-1,
        **best_trial.params,
    )
    final_model.fit(X, y, categorical_feature=categorical_features)

    # print rmse on the training set
    train_preds = final_model.predict(X)
    train_rmse = np.sqrt(mean_squared_error(y, train_preds))
    print(f"\nFinal Model Training RMSE: {train_rmse:.4f}")

    # Show feature importances
    try:
        lgb.plot_importance(final_model, max_num_features=20)
        import matplotlib.pyplot as plt

        plt.title("Feature Importances")
    except Exception as e:
        print(
            f"\nCould not plot feature importance. (Is matplotlib installed?) Error: {e}"
        )

In [18]:
# nécessité de faire du feature engineering i.e. prendre des moyennes ou qqch parce que là ce n'est pas possible
# example approche actuelle: premier prompt de https://gemini.google.com/app/2db88ca378891ac5

In [19]:
flights, traj, fuel, airports = load_data()

Loading data...
Loaded flight list, fuel, and airport data.
Found 5868 trajectory files. Loading and concatenating...
Trajectory data concatenated successfully.
Preprocessing data...


In [20]:
# print the number of rows in each dataframe
print(f"Flights: {len(flights)} rows")
print(f"Trajectories: {len(traj)} rows")
print(f"Fuel: {len(fuel)} rows")
print(f"Airports: {len(airports)} rows")

Flights: 11037 rows
Trajectories: 65550915 rows
Fuel: 131530 rows
Airports: 8787 rows


In [21]:
# print the list of aircraft types by number of occurrences
print("Aircraft types in the dataset:")
print(flights["aircraft_type"].value_counts())

Aircraft types in the dataset:
aircraft_type
A20N    3220
A320    3023
A359    1571
B788     583
B738     551
A332     532
A21N     393
A321     285
B789     224
B77W     183
A333     156
B772     103
B744      90
B737      29
B739      28
B38M      20
A319      19
A306      10
A388       4
B748       3
B752       3
B763       2
B77L       2
MD11       1
A318       1
B39M       1
Name: count, dtype: int64


In [32]:
# keep only one type of aircraft for faster testing
flights_B772 = flights[flights["aircraft_type"] == "B772"].reset_index(drop=True)
traj_B772 = traj[traj["flight_id"].isin(flights_B772["flight_id"])].reset_index(
    drop=True
)
fuel_B772 = fuel[fuel["flight_id"].isin(flights_B772["flight_id"])].reset_index(
    drop=True
)
airports_B772 = airports.copy()
display(flights_B772)

,flight_date,aircraft_type,takeoff,landed,origin_icao,origin_name,destination_icao,destination_name,flight_id
0,1744675200000000000,B772,2025-04-15 00:08:19,2025-04-15 10:02:38,KLAX,Los Angeles International Airport,EHAM,Amsterdam Schiphol Airport,prc771357721
1,1744848000000000000,B772,2025-04-17 11:08:57,2025-04-17 22:35:12,EHAM,Amsterdam Schiphol Airport,SBGL,Rio de Janeiro Galeao International Airport,prc771965145
2,1744934400000000000,B772,2025-04-18 01:08:02,2025-04-18 11:34:06,SBGL,Rio de Janeiro Galeao International Airport,EHAM,Amsterdam Schiphol Airport,prc772142397
3,1745020800000000000,B772,2025-04-19 00:43:29,2025-04-19 10:25:20,KLAX,Los Angeles International Airport,EHAM,Amsterdam Schiphol Airport,prc772377375
4,1745020800000000000,B772,2025-04-19 12:47:52,2025-04-20 01:26:11,EHAM,Amsterdam Schiphol Airport,RJAA,Tokyo Narita International Airport,prc772480640
...,...,...,...,...,...,...,...,...,...
98,1753574400000000000,B772,2025-07-27 08:38:00,2025-07-27 18:35:16,EHAM,Amsterdam Schiphol Airport,KIAH,Houston George Bush Intercontinental Airport,prc797320529
99,1754006400000000000,B772,2025-08-01 03:50:21,2025-08-01 13:13:29,KLAS,Las Vegas Harry Reid International Airport,EHAM,Amsterdam Schiphol Airport,prc798587369
100,1754438400000000000,B772,2025-08-06 08:33:31,2025-08-06 18:18:23,EHAM,Amsterdam Schiphol Airport,KIAH,Houston George Bush Intercontinental Airport,prc799902672
101,1754438400000000000,B772,2025-08-06 15:28:10,2025-08-07 04:11:58,VHHH,Hong Kong International Airport,EHAM,Amsterdam Schiphol Airport,prc799995414


In [33]:
X_features, y_target, cat_features = create_features(
    flights_B772, traj_B772, fuel_B772, airports_B772, to_display=True
)

print(len(X_features), "samples in the feature set")

Engineering features...
Aggregating trajectory features...
Finalizing dataset...
Dataset finalized. Total samples: 513
Features: ['interval_duration_s', 'aircraft_type', 'origin_elevation', 'dest_elevation', 'mean_altitude', 'max_altitude', 'mean_TAS', 'max_TAS', 'mean_groundspeed', 'mean_vertical_rate', 'n_points', 'altitude_change', 'time_in_interval_s']


/tmp/ipykernel_21147/1316261788.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = (


,interval_duration_s,aircraft_type,origin_elevation,dest_elevation,mean_altitude,max_altitude,mean_TAS,max_TAS,mean_groundspeed,mean_vertical_rate,n_points,altitude_change,time_in_interval_s
24,1806.758,B772,28.0,-11.0,22451.01767,31024.9824,0.0,0.0,428.315306,923.62449,2239.0,30825.0336,1806.758
25,892.171,B772,28.0,-11.0,30999.95748,30999.9888,0.0,0.0,497.253394,0.00000,1598.0,0.0000,892.171
26,2707.764,B772,28.0,-11.0,30999.98880,30999.9888,0.0,0.0,493.007752,0.00000,331.0,0.0000,2707.764
27,3592.354,B772,28.0,-11.0,0.00000,0.0000,0.0,0.0,0.000000,0.00000,2.0,0.0000,3592.354
28,911.596,B772,28.0,-11.0,0.00000,0.0000,0.0,0.0,0.000000,0.00000,2.0,0.0000,911.596


24    7000.0
25    2000.0
26    6300.0
27    7900.0
28    2000.0
Name: fuel_kg, dtype: float64

Categorical features: ['aircraft_type']
513 samples in the feature set


In [34]:
model_name = train_model(
    X_features,
    y_target,
    cat_features,
    n_trials=50,
    # to_load="LightGBM Fuel Burn Prediction 2025-11-04T13:18:37.793915",
)
# to load the dashboard do: uv run optuna-dashboard sqlite:///mydb.sqlite3
test_model(
    X_features,
    y_target,
    cat_features,
    model_to_load=model_name,
    storage="sqlite:///mydb.sqlite3",
)

[I 2025-11-04 13:49:48,874] A new study created in RDB with name: LightGBM Fuel Burn Prediction 2025-11-04T13:49:48.781541



--- Starting Optuna Hyperparameter Search ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-04 13:50:53,693] Trial 0 finished with value: 475.8676715264503 and parameters: {'learning_rate': 0.1850283420831427, 'num_leaves': 129, 'max_depth': 18, 'subsample': 0.8835050529508559, 'colsample_bytree': 0.8298347640292779, 'reg_alpha': 0.2191450909921263, 'reg_lambda': 0.10020902857504332}. Best is trial 0 with value: 475.8676715264503.
[I 2025-11-04 13:51:55,799] Trial 1 finished with value: 445.43803576776446 and parameters: {'learning_rate': 0.15211866968953558, 'num_leaves': 228, 'max_depth': 13, 'subsample': 0.9211923813142932, 'colsample_bytree': 0.8438068703777372, 'reg_alpha': 0.49013380147214447, 'reg_lambda': 0.450802947808142}. Best is trial 1 with value: 445.43803576776446.
[I 2025-11-04 13:52:58,740] Trial 2 finished with value: 449.5537698498947 and parameters: {'learning_rate': 0.27959698815447315, 'num_leaves': 237, 'max_depth': 19, 'subsample': 0.6175695712237393, 'colsample_bytree': 0.9716633618922775, 'reg_alpha': 0.4793774631285168, 'reg_lambda': 0.39

KeyboardInterrupt: 

In [ ]:
# # keep only the first 150 rows of fuel for faster testing (130k rows is a bit too much)
# fuel_short = fuel.iloc[:150].reset_index(drop=True)

# X_features, y_target, categorical_cols = create_features(flights, traj, fuel_short, airports, to_display=True)

# print(len(X_features), "samples in the feature set")

In [26]:
# model_name = train_model(X_features, y_target, categorical_cols, n_trials=100)
# # to load the dashboard do: uv run optuna-dashboard sqlite:///mydb.sqlite3
# test_model(
#     X_features,
#     y_target,
#     categorical_cols,
#     model_to_load=model_name,
#     storage="sqlite:///mydb.sqlite3",
# )